<a href="https://colab.research.google.com/github/dlrstrum/labs/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Race

import pygame, sys
from pygame.locals import *
import random, time

pygame.init()

# Game settings
FPS = 60
FramePerSec = pygame.time.Clock()
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
coinscore = 0
COIN_SPEED = 5
COIN_THRESHOLD = 5  # Coins required to increase enemy speed
ENEMY_SPEED_INCREMENT = 1
COIN_SIZE = (60, 60)  # Standard coin size

# Load assets
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, "black")
background = pygame.image.load(r"C:\projects\pp2\Lab9\Source\road (1).png")
coin_images = {
    1: pygame.transform.scale(pygame.image.load(r"C:\projects\pp2\Lab9\Source\coin1.png"), COIN_SIZE),
    2: pygame.transform.scale(pygame.image.load(r"C:\projects\pp2\Lab9\Source\coin2.jpg"), COIN_SIZE),
    3: pygame.transform.scale(pygame.image.load(r"C:\projects\pp2\Lab9\Source\coin3.jpg"), COIN_SIZE),
}

# Display setup
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Racer")

# Enemy class
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.transform.scale(pygame.image.load(r"C:\projects\pp2\Lab9\Source\Enemy (1).png"), (55, 110))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Player class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.transform.scale(pygame.image.load(r"C:\projects\pp2\Lab9\Source\Player.png"), (60, 120))
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-7, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:
            self.rect.move_ip(7, 0)

# Coin class with different weights
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.value = random.choice([1, 2, 3])  # Random coin value
        self.image = coin_images[self.value]  # Assign scaled image based on value
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(32, SCREEN_WIDTH-32), -31)

    def move(self):
        global coinscore, SPEED
        self.rect.move_ip(0, COIN_SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            self.reset_position()
        elif self.rect.colliderect(P1.rect):
            coinscore += self.value  # Increase score by coin value
            if coinscore % COIN_THRESHOLD == 0:
                increase_enemy_speed()
            self.reset_position()

    def reset_position(self):
        self.value = random.choice([1, 2, 3])  # Assign new random value
        self.image = coin_images[self.value]  # Assign corresponding scaled image
        self.rect.top = -31
        self.rect.center = (random.randint(32, SCREEN_WIDTH-32), -31)

# Increase enemy speed function
def increase_enemy_speed():
    global SPEED
    SPEED += ENEMY_SPEED_INCREMENT

# Initialize objects
P1 = Player()
E1 = Enemy()
C1 = Coin()

# Sprite groups
enemies = pygame.sprite.Group()
enemies.add(E1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1)
coins = pygame.sprite.Group()
coins.add(C1)

# Background scrolling variables
bgy = 0
bgsound = pygame.mixer.Sound(r"C:\projects\pp2\Lab9\Source\background.wav")
bgsound.play(loops=-1)

# Game loop
while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Scroll background
    DISPLAYSURF.blit(background, (0, bgy))
    DISPLAYSURF.blit(background, (0, bgy-600))
    bgy = (bgy + 7) % 600

    # Display scores
    scores = font_small.render(str(SCORE), True, "BLACK")
    coinscores = font_small.render(str(coinscore), True, "BLACK")
    DISPLAYSURF.blit(scores, (10, 10))
    DISPLAYSURF.blit(coinscores, (360, 10))

    # Move and draw all sprites
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move()
    for coin in coins:
        DISPLAYSURF.blit(coin.image, coin.rect)
        coin.move()

    # Collision detection
    if pygame.sprite.spritecollideany(P1, enemies):
        bgsound.stop()
        pygame.mixer.Sound(r"C:\projects\pp2\Lab9\Source\crash.wav").play()
        time.sleep(0.5)
        DISPLAYSURF.fill("RED")
        DISPLAYSURF.blit(game_over, (30, 250))
        pygame.display.update()
        time.sleep(2)
        pygame.quit()
        sys.exit()

    pygame.display.update()
    FramePerSec.tick(FPS)


In [ ]:
#Snake

import pygame
import random

# Инициализация Pygame
pygame.init()

# Размеры окна и сетки
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GRID_SIZE = 20
GRID_WIDTH = SCREEN_WIDTH // GRID_SIZE
GRID_HEIGHT = SCREEN_HEIGHT // GRID_SIZE
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Snake Game')

# Цвета
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
BLACK = (0, 0, 0)
DARK_RED = (139, 0, 0)

# Направления движения змейки
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

# Класс змейки
class Snake:
    def __init__(self):
        self.length = 1
        self.positions = [(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.color = GREEN
        self.alive = True

    def get_head_position(self):
        return self.positions[0]

    def move(self):
        if not self.alive:
            return

        head_x, head_y = self.get_head_position()
        dx, dy = self.direction
        new_head = (head_x + dx * GRID_SIZE, head_y + dy * GRID_SIZE)

        # Проверка столкновений
        if new_head[0] < 0 or new_head[0] >= SCREEN_WIDTH or new_head[1] < 0 or new_head[1] >= SCREEN_HEIGHT:
            self.alive = False
            return

        if new_head in self.positions:
            self.alive = False
            return

        self.positions.insert(0, new_head)
        if len(self.positions) > self.length:
            self.positions.pop()

    def reset(self):
        self.__init__()

    def draw(self, surface):
        for segment in self.positions:
            pygame.draw.rect(surface, self.color, pygame.Rect(segment[0], segment[1], GRID_SIZE, GRID_SIZE))
            pygame.draw.rect(surface, WHITE, pygame.Rect(segment[0], segment[1], GRID_SIZE, GRID_SIZE), 1)

    def handle_keys(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and self.direction != DOWN:
                    self.direction = UP
                elif event.key == pygame.K_DOWN and self.direction != UP:
                    self.direction = DOWN
                elif event.key == pygame.K_LEFT and self.direction != RIGHT:
                    self.direction = LEFT
                elif event.key == pygame.K_RIGHT and self.direction != LEFT:
                    self.direction = RIGHT

# Класс еды с разными типами и таймером
class Food:
    def __init__(self):
        self.food_types = [
            {'color': RED, 'score': 1, 'time': 300},  # Обычная еда
            {'color': BLUE, 'score': 3, 'time': 200},  # Дает больше очков
            {'color': YELLOW, 'score': 5, 'time': 150}  # Дает много очков, но исчезает быстрее
        ]
        self.type = random.choice(self.food_types)
        self.position = (0, 0)
        self.timer = self.type['time']
        self.randomize_position([])

    def randomize_position(self, snake_positions):
        while True:
            new_position = (random.randint(0, GRID_WIDTH - 1) * GRID_SIZE,
                            random.randint(0, GRID_HEIGHT - 1) * GRID_SIZE)
            if new_position not in snake_positions:
                self.position = new_position
                self.type = random.choice(self.food_types)  # Меняем тип еды при генерации
                self.timer = self.type['time']
                break

    def update(self):
        self.timer -= 1
        return self.timer > 0

    def draw(self, surface):
        pygame.draw.rect(surface, self.type['color'], pygame.Rect(self.position[0], self.position[1], GRID_SIZE, GRID_SIZE))
        pygame.draw.rect(surface, WHITE, pygame.Rect(self.position[0], self.position[1], GRID_SIZE, GRID_SIZE), 1)

# Функции отрисовки текста и экрана смерти
def draw_text(surface, text, x, y, size=36, color=WHITE):
    font = pygame.font.Font(None, size)
    text_surface = font.render(text, True, color)
    surface.blit(text_surface, (x, y))

def game_over_screen(score):
    screen.fill(DARK_RED)
    draw_text(screen, "ВЫ УМЕРЛИ!", SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 - 50, 50, WHITE)
    draw_text(screen, f"Ваш счет: {score}", SCREEN_WIDTH // 2 - 80, SCREEN_HEIGHT // 2, 40, WHITE)
    draw_text(screen, "Нажмите ПРОБЕЛ для рестарта или ESC для выхода", SCREEN_WIDTH // 2 - 250, SCREEN_HEIGHT // 2 + 50, 30, WHITE)
    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    exit()

# Основной игровой цикл
def main():
    while True:
        snake = Snake()
        food = Food()
        score = 0
        level = 1
        clock = pygame.time.Clock()

        while snake.alive:
            screen.fill(BLACK)
            snake.handle_keys()
            snake.move()

            if snake.get_head_position() == food.position:
                snake.length += 1
                score += food.type['score']
                food.randomize_position(snake.positions)

                if score % 3 == 0:
                    level += 1

            if not food.update():
                food.randomize_position(snake.positions)

            if not snake.alive:
                game_over_screen(score)
                break

            snake.draw(screen)
            food.draw(screen)
            draw_text(screen, f"Score: {score}", 10, 10)
            draw_text(screen, f"Level: {level}", 10, 40)

            pygame.display.update()
            clock.tick(8 + level)

if __name__ == '__main__':
    main()


In [ ]:
#Paint

import pygame
import math

# Инициализация Pygame
pygame.init()

# Размеры экрана
WIDTH, HEIGHT = 900, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("GFG Paint")

# Цвета
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (128, 0, 128)

# Задний фон (белый)
screen.fill(WHITE)

# Кнопки выбора цветов
COLOR_BUTTONS = [
    {"color": RED, "rect": pygame.Rect(10, 10, 50, 50)},
    {"color": YELLOW, "rect": pygame.Rect(70, 10, 50, 50)},
    {"color": GREEN, "rect": pygame.Rect(130, 10, 50, 50)},
    {"color": BLUE, "rect": pygame.Rect(190, 10, 50, 50)},
    {"color": PURPLE, "rect": pygame.Rect(250, 10, 50, 50)}
]

# Загрузка изображения ластика
eraser_img = pygame.image.load('C:\projects\pp2\Lab9\Source/eraser.png')
eraser_img = pygame.transform.scale(eraser_img, (50, 50))

# Кнопки выбора фигур
TOOLS = [
    {"name": "Линия", "rect": pygame.Rect(320, 10, 90, 40), "type": "pencil"},
    {"name": "Прямоугольник", "rect": pygame.Rect(420, 10, 150, 40), "type": "rectangle"},
    {"name": "Круг", "rect": pygame.Rect(580, 10, 90, 40), "type": "circle"},
    {"name": "Квадрат", "rect": pygame.Rect(680, 10, 90, 40), "type": "square"},
    {"name": "Пр. Треугольник", "rect": pygame.Rect(780, 10, 120, 40), "type": "right_triangle"},
    {"name": "Равн. Треугольник", "rect": pygame.Rect(320, 60, 150, 40), "type": "equilateral_triangle"},
    {"name": "Ромб", "rect": pygame.Rect(480, 60, 90, 40), "type": "rhombus"},
]

# Кнопка выбора ластика
eraser_button = pygame.Rect(580, 60, 50, 50)

# Текущие настройки
current_color = BLACK
current_tool = "pencil"
drawing = False
start_pos = None
last_pos = None

# Функции для рисования фигур
def draw_pencil(surface, color, start, end, width=3):
    pygame.draw.line(surface, color, start, end, width)

def draw_rectangle(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    pygame.draw.rect(surface, color, (min(x1, x2), min(y1, y2), width, height), 3)

def draw_circle(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    radius = int(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))
    pygame.draw.circle(surface, color, start, radius, 3)

def draw_square(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    side = min(abs(x2 - x1), abs(y2 - y1))
    pygame.draw.rect(surface, color, (x1, y1, side, side), 3)

def draw_right_triangle(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    points = [(x1, y1), (x1, y2), (x2, y2)]
    pygame.draw.polygon(surface, color, points, 3)

def draw_equilateral_triangle(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    side = abs(x2 - x1)
    height = int(math.sqrt(3) / 2 * side)
    points = [(x1, y1 + height), (x1 + side, y1 + height), (x1 + side // 2, y1)]
    pygame.draw.polygon(surface, color, points, 3)

def draw_rhombus(surface, color, start, end):
    x1, y1 = start
    x2, y2 = end
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    points = [(x1 + width // 2, y1), (x2, y1 + height // 2), (x1 + width // 2, y2), (x1, y1 + height // 2)]
    pygame.draw.polygon(surface, color, points, 3)

def erase_area(surface, position):
    pygame.draw.circle(surface, WHITE, position, 20)

# Основной цикл программы
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                # Проверка кнопок инструментов
                for button in TOOLS:
                    if button["rect"].collidepoint(event.pos):
                        current_tool = button["type"]

                # Проверка кнопки ластика
                if eraser_button.collidepoint(event.pos):
                    current_tool = "eraser"

                # Проверка кнопок цветов
                for button in COLOR_BUTTONS:
                    if button["rect"].collidepoint(event.pos):
                        current_color = button["color"]

                start_pos = event.pos
                last_pos = event.pos
                drawing = True

        if event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                drawing = False
                end_pos = event.pos
                if start_pos and end_pos:
                    shapes = {
                        "rectangle": draw_rectangle,
                        "circle": draw_circle,
                        "square": draw_square,
                        "right_triangle": draw_right_triangle,
                        "equilateral_triangle": draw_equilateral_triangle,
                        "rhombus": draw_rhombus,
                    }
                    if current_tool in shapes:
                        shapes[current_tool](screen, current_color, start_pos, end_pos)
                        pygame.display.update()  # Обновление экрана после рисования

        # Движение мыши для рисования
        if event.type == pygame.MOUSEMOTION:
            if drawing:
                if current_tool == "pencil":
                    draw_pencil(screen, current_color, last_pos, event.pos, 3)
                    last_pos = event.pos  # Обновление последней позиции
                elif current_tool == "eraser":
                    erase_area(screen, event.pos)

    # Рисуем кнопки цветов
    for button in COLOR_BUTTONS:
        pygame.draw.rect(screen, button["color"], button["rect"])

    # Рисуем кнопки инструментов
    for button in TOOLS:
        pygame.draw.rect(screen, BLACK, button["rect"], 2)
        font = pygame.font.Font(None, 24)
        text = font.render(button["name"], True, BLACK)
        screen.blit(text, (button["rect"].x + 5, button["rect"].y + 10))

    # Кнопка ластика
    pygame.draw.rect(screen, BLACK, eraser_button, 2)
    screen.blit(eraser_img, (eraser_button.x, eraser_button.y))

    pygame.display.flip()

pygame.quit()
